In [1]:
import argparse
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
from mobilenets import mobilenet

In [2]:
use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [3]:
# Train, Validate, Test. Heavily inspired by Kevinzakka https://github.com/kevinzakka/DenseNet/blob/master/data_loader.py

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

valid_size=0.1

# define transforms
valid_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])


# load the dataset
train_dataset = datasets.CIFAR10(root="/home/paperspace/Documents/GitHub/data", train=True, 
            download=True, transform=train_transform)

valid_dataset = datasets.CIFAR10(root="/home/paperspace/Documents/GitHub/data", train=True, 
            download=True, transform=valid_transform)

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))


np.random.seed(42)
np.random.shuffle(indices)

train_idx, valid_idx = indices[split:], indices[:split]


train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)


train_loader = torch.utils.data.DataLoader(train_dataset, 
                batch_size=64, sampler=train_sampler)

valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                batch_size=64, sampler=valid_sampler)


test_transform = transforms.Compose([
    transforms.ToTensor(), normalize
])

test_dataset = datasets.CIFAR10(root="/home/paperspace/Documents/GitHub/data", 
                                train=False, 
                                download=True,transform=test_transform)

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=64, 
                                          shuffle=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
# class Alex_Net(nn.Module):
#     def __init__(self, num_classes=10):
#         super(Alex_Net, self).__init__()
#         self.features = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=5),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(64, 192, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#             nn.Conv2d(192, 384, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(384, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(kernel_size=2, stride=2),
#         )
#         self.classifier = nn.Linear(256, num_classes)

#     def forward(self, x):
#         x = self.features(x)
#         x = x.view(x.size(0), -1)
#         x = self.classifier(x)
#         return F.log_softmax(x)
    
# def alex_net():
#     model = Alex_Net()
#     if use_cuda:
#         model = model.cuda()
#     return model

# model = alex_net()
# optimizer = optim.Adam(model.parameters())

# print(model)

In [5]:
model = mobilenet(num_classes=10, large_img=False)
optimizer = optim.Adam(model.parameters(), lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

In [6]:
# Implement validation

def train(epoch):
    model.train()
    writer = SummaryWriter()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        correct = 0
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()
        loss = criterion(output, target)
        loss.backward()
        accuracy = 100. * (correct/ len(output))
        optimizer.step()
        if batch_idx % 1 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0], accuracy))
            writer.add_scalar('Loss/Loss', loss.data[0], epoch)
            writer.add_scalar('Accuracy/Accuracy', accuracy, epoch)
    scheduler.step()

In [7]:
def validate(epoch):
    model.eval()
    writer = SummaryWriter()
    valid_loss = 0
    correct = 0
    for data, target in valid_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        valid_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).sum()

    valid_loss /= len(valid_idx)
    accuracy = 100. * correct / len(valid_idx)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        valid_loss, correct, len(valid_idx),
        100. * correct / len(valid_idx)))
    writer.add_scalar('Loss/Validation_Loss', valid_loss, epoch)
    writer.add_scalar('Accuracy/Validation_Accuracy', accuracy, epoch)
    return valid_loss, accuracy
    

In [8]:
# Fix best model

def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
# Tensor board data logger

In [10]:
def save_best(loss, accuracy, best_loss, best_acc):
    if best_loss == None:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    elif loss < best_loss and accuracy > best_acc:
        best_loss = loss
        best_acc = accuracy
        file = 'saved_models/best_save_model.p'
        torch.save(model.state_dict(), file)
    return best_loss, best_acc

In [11]:
# Fantastic logger for tensorboard and pytorch, 
# run tensorboard by opening a new terminal and run "tensorboard --logdir runs"
# open tensorboard at http://localhost:6006/
from tensorboardX import SummaryWriter
best_loss = None
best_acc = None

for epoch in range(50):
    train(epoch)
    loss, accuracy = validate(epoch)
    best_loss, best_acc = save_best(loss, accuracy, best_loss, best_acc)
    
writer.export_scalars_to_json("./all_scalars.json")

writer.close()

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.320784, Accuracy: 4.69
Train Epoch: 0 [64/50000 (0%)]	Loss: 2.993663, Accuracy: 6.25
Train Epoch: 0 [128/50000 (0%)]	Loss: 3.182273, Accuracy: 3.12
Train Epoch: 0 [192/50000 (0%)]	Loss: 2.403641, Accuracy: 9.38
Train Epoch: 0 [256/50000 (1%)]	Loss: 2.448259, Accuracy: 10.94
Train Epoch: 0 [320/50000 (1%)]	Loss: 2.981651, Accuracy: 9.38
Train Epoch: 0 [384/50000 (1%)]	Loss: 2.594281, Accuracy: 6.25
Train Epoch: 0 [448/50000 (1%)]	Loss: 2.776117, Accuracy: 9.38
Train Epoch: 0 [512/50000 (1%)]	Loss: 2.803042, Accuracy: 10.94
Train Epoch: 0 [576/50000 (1%)]	Loss: 2.364301, Accuracy: 17.19
Train Epoch: 0 [640/50000 (1%)]	Loss: 2.579102, Accuracy: 10.94
Train Epoch: 0 [704/50000 (2%)]	Loss: 2.385206, Accuracy: 12.50
Train Epoch: 0 [768/50000 (2%)]	Loss: 2.400055, Accuracy: 18.75
Train Epoch: 0 [832/50000 (2%)]	Loss: 2.476136, Accuracy: 10.94
Train Epoch: 0 [896/50000 (2%)]	Loss: 2.361637, Accuracy: 18.75
Train Epoch: 0 [960/50000 (2%)]	Loss: 2.727899, Ac

Train Epoch: 0 [8064/50000 (18%)]	Loss: 1.891530, Accuracy: 31.25
Train Epoch: 0 [8128/50000 (18%)]	Loss: 1.983203, Accuracy: 29.69
Train Epoch: 0 [8192/50000 (18%)]	Loss: 1.988982, Accuracy: 26.56
Train Epoch: 0 [8256/50000 (18%)]	Loss: 2.002601, Accuracy: 15.62
Train Epoch: 0 [8320/50000 (18%)]	Loss: 1.918661, Accuracy: 28.12
Train Epoch: 0 [8384/50000 (19%)]	Loss: 1.905090, Accuracy: 31.25
Train Epoch: 0 [8448/50000 (19%)]	Loss: 2.033516, Accuracy: 23.44
Train Epoch: 0 [8512/50000 (19%)]	Loss: 1.914131, Accuracy: 21.88
Train Epoch: 0 [8576/50000 (19%)]	Loss: 2.035458, Accuracy: 20.31
Train Epoch: 0 [8640/50000 (19%)]	Loss: 2.226429, Accuracy: 20.31
Train Epoch: 0 [8704/50000 (19%)]	Loss: 2.086241, Accuracy: 23.44
Train Epoch: 0 [8768/50000 (19%)]	Loss: 1.879201, Accuracy: 23.44
Train Epoch: 0 [8832/50000 (20%)]	Loss: 1.918816, Accuracy: 21.88
Train Epoch: 0 [8896/50000 (20%)]	Loss: 1.978667, Accuracy: 17.19
Train Epoch: 0 [8960/50000 (20%)]	Loss: 1.943534, Accuracy: 26.56
Train Epoc

Train Epoch: 0 [15936/50000 (35%)]	Loss: 2.034458, Accuracy: 21.88
Train Epoch: 0 [16000/50000 (36%)]	Loss: 1.930669, Accuracy: 20.31
Train Epoch: 0 [16064/50000 (36%)]	Loss: 2.028608, Accuracy: 17.19
Train Epoch: 0 [16128/50000 (36%)]	Loss: 1.896047, Accuracy: 26.56
Train Epoch: 0 [16192/50000 (36%)]	Loss: 1.977671, Accuracy: 17.19
Train Epoch: 0 [16256/50000 (36%)]	Loss: 1.715361, Accuracy: 34.38
Train Epoch: 0 [16320/50000 (36%)]	Loss: 1.888401, Accuracy: 25.00
Train Epoch: 0 [16384/50000 (36%)]	Loss: 1.890301, Accuracy: 31.25
Train Epoch: 0 [16448/50000 (37%)]	Loss: 2.036516, Accuracy: 21.88
Train Epoch: 0 [16512/50000 (37%)]	Loss: 1.809470, Accuracy: 29.69
Train Epoch: 0 [16576/50000 (37%)]	Loss: 2.000942, Accuracy: 29.69
Train Epoch: 0 [16640/50000 (37%)]	Loss: 1.851443, Accuracy: 25.00
Train Epoch: 0 [16704/50000 (37%)]	Loss: 1.960647, Accuracy: 21.88
Train Epoch: 0 [16768/50000 (37%)]	Loss: 1.954419, Accuracy: 25.00
Train Epoch: 0 [16832/50000 (37%)]	Loss: 1.766016, Accuracy: 3

Train Epoch: 0 [23808/50000 (53%)]	Loss: 1.996032, Accuracy: 28.12
Train Epoch: 0 [23872/50000 (53%)]	Loss: 1.927146, Accuracy: 28.12
Train Epoch: 0 [23936/50000 (53%)]	Loss: 1.876081, Accuracy: 40.62
Train Epoch: 0 [24000/50000 (53%)]	Loss: 1.679407, Accuracy: 39.06
Train Epoch: 0 [24064/50000 (53%)]	Loss: 1.863056, Accuracy: 34.38
Train Epoch: 0 [24128/50000 (54%)]	Loss: 1.768157, Accuracy: 32.81
Train Epoch: 0 [24192/50000 (54%)]	Loss: 1.794794, Accuracy: 32.81
Train Epoch: 0 [24256/50000 (54%)]	Loss: 1.943245, Accuracy: 25.00
Train Epoch: 0 [24320/50000 (54%)]	Loss: 2.005270, Accuracy: 31.25
Train Epoch: 0 [24384/50000 (54%)]	Loss: 1.872349, Accuracy: 26.56
Train Epoch: 0 [24448/50000 (54%)]	Loss: 1.884419, Accuracy: 31.25
Train Epoch: 0 [24512/50000 (54%)]	Loss: 1.876156, Accuracy: 32.81
Train Epoch: 0 [24576/50000 (55%)]	Loss: 1.657971, Accuracy: 35.94
Train Epoch: 0 [24640/50000 (55%)]	Loss: 1.802757, Accuracy: 37.50
Train Epoch: 0 [24704/50000 (55%)]	Loss: 1.786251, Accuracy: 3

Train Epoch: 0 [31680/50000 (70%)]	Loss: 1.826103, Accuracy: 23.44
Train Epoch: 0 [31744/50000 (70%)]	Loss: 1.790051, Accuracy: 35.94
Train Epoch: 0 [31808/50000 (71%)]	Loss: 1.795999, Accuracy: 35.94
Train Epoch: 0 [31872/50000 (71%)]	Loss: 1.763695, Accuracy: 31.25
Train Epoch: 0 [31936/50000 (71%)]	Loss: 1.762780, Accuracy: 28.12
Train Epoch: 0 [32000/50000 (71%)]	Loss: 1.805178, Accuracy: 31.25
Train Epoch: 0 [32064/50000 (71%)]	Loss: 1.764626, Accuracy: 35.94
Train Epoch: 0 [32128/50000 (71%)]	Loss: 1.823982, Accuracy: 35.94
Train Epoch: 0 [32192/50000 (71%)]	Loss: 1.584934, Accuracy: 45.31
Train Epoch: 0 [32256/50000 (72%)]	Loss: 1.754930, Accuracy: 34.38
Train Epoch: 0 [32320/50000 (72%)]	Loss: 1.772480, Accuracy: 28.12
Train Epoch: 0 [32384/50000 (72%)]	Loss: 1.699659, Accuracy: 29.69
Train Epoch: 0 [32448/50000 (72%)]	Loss: 1.762239, Accuracy: 34.38
Train Epoch: 0 [32512/50000 (72%)]	Loss: 1.764101, Accuracy: 29.69
Train Epoch: 0 [32576/50000 (72%)]	Loss: 1.722189, Accuracy: 3

...

Train Epoch: 49 [14272/50000 (32%)]	Loss: 0.250722, Accuracy: 92.19
Train Epoch: 49 [14336/50000 (32%)]	Loss: 0.203114, Accuracy: 93.75
Train Epoch: 49 [14400/50000 (32%)]	Loss: 0.134134, Accuracy: 93.75
Train Epoch: 49 [14464/50000 (32%)]	Loss: 0.152649, Accuracy: 92.19
Train Epoch: 49 [14528/50000 (32%)]	Loss: 0.184675, Accuracy: 90.62
Train Epoch: 49 [14592/50000 (32%)]	Loss: 0.113539, Accuracy: 96.88
Train Epoch: 49 [14656/50000 (33%)]	Loss: 0.121432, Accuracy: 96.88
Train Epoch: 49 [14720/50000 (33%)]	Loss: 0.159186, Accuracy: 93.75
Train Epoch: 49 [14784/50000 (33%)]	Loss: 0.275920, Accuracy: 87.50
Train Epoch: 49 [14848/50000 (33%)]	Loss: 0.207045, Accuracy: 93.75
Train Epoch: 49 [14912/50000 (33%)]	Loss: 0.330669, Accuracy: 90.62
Train Epoch: 49 [14976/50000 (33%)]	Loss: 0.243859, Accuracy: 89.06
Train Epoch: 49 [15040/50000 (33%)]	Loss: 0.168357, Accuracy: 93.75
Train Epoch: 49 [15104/50000 (34%)]	Loss: 0.321262, Accuracy: 85.94
Train Epoch: 49 [15168/50000 (34%)]	Loss: 0.2984

Train Epoch: 49 [22016/50000 (49%)]	Loss: 0.239828, Accuracy: 92.19
Train Epoch: 49 [22080/50000 (49%)]	Loss: 0.231717, Accuracy: 92.19
Train Epoch: 49 [22144/50000 (49%)]	Loss: 0.299078, Accuracy: 89.06
Train Epoch: 49 [22208/50000 (49%)]	Loss: 0.103353, Accuracy: 96.88
Train Epoch: 49 [22272/50000 (49%)]	Loss: 0.289471, Accuracy: 92.19
Train Epoch: 49 [22336/50000 (50%)]	Loss: 0.114347, Accuracy: 92.19
Train Epoch: 49 [22400/50000 (50%)]	Loss: 0.313370, Accuracy: 87.50
Train Epoch: 49 [22464/50000 (50%)]	Loss: 0.416033, Accuracy: 87.50
Train Epoch: 49 [22528/50000 (50%)]	Loss: 0.275740, Accuracy: 89.06
Train Epoch: 49 [22592/50000 (50%)]	Loss: 0.172632, Accuracy: 92.19
Train Epoch: 49 [22656/50000 (50%)]	Loss: 0.135407, Accuracy: 95.31
Train Epoch: 49 [22720/50000 (50%)]	Loss: 0.147171, Accuracy: 95.31
Train Epoch: 49 [22784/50000 (51%)]	Loss: 0.220210, Accuracy: 92.19
Train Epoch: 49 [22848/50000 (51%)]	Loss: 0.132306, Accuracy: 95.31
Train Epoch: 49 [22912/50000 (51%)]	Loss: 0.2478

Train Epoch: 49 [29760/50000 (66%)]	Loss: 0.314517, Accuracy: 85.94
Train Epoch: 49 [29824/50000 (66%)]	Loss: 0.042652, Accuracy: 100.00
Train Epoch: 49 [29888/50000 (66%)]	Loss: 0.281088, Accuracy: 92.19
Train Epoch: 49 [29952/50000 (66%)]	Loss: 0.180408, Accuracy: 95.31
Train Epoch: 49 [30016/50000 (67%)]	Loss: 0.106700, Accuracy: 96.88
Train Epoch: 49 [30080/50000 (67%)]	Loss: 0.203858, Accuracy: 87.50
Train Epoch: 49 [30144/50000 (67%)]	Loss: 0.713731, Accuracy: 79.69
Train Epoch: 49 [30208/50000 (67%)]	Loss: 0.265067, Accuracy: 90.62
Train Epoch: 49 [30272/50000 (67%)]	Loss: 0.188904, Accuracy: 93.75
Train Epoch: 49 [30336/50000 (67%)]	Loss: 0.194041, Accuracy: 92.19
Train Epoch: 49 [30400/50000 (67%)]	Loss: 0.241392, Accuracy: 90.62
Train Epoch: 49 [30464/50000 (68%)]	Loss: 0.140680, Accuracy: 95.31
Train Epoch: 49 [30528/50000 (68%)]	Loss: 0.274847, Accuracy: 90.62
Train Epoch: 49 [30592/50000 (68%)]	Loss: 0.199456, Accuracy: 89.06
Train Epoch: 49 [30656/50000 (68%)]	Loss: 0.217

Train Epoch: 49 [37504/50000 (83%)]	Loss: 0.135015, Accuracy: 96.88
Train Epoch: 49 [37568/50000 (83%)]	Loss: 0.117836, Accuracy: 96.88
Train Epoch: 49 [37632/50000 (84%)]	Loss: 0.245364, Accuracy: 92.19
Train Epoch: 49 [37696/50000 (84%)]	Loss: 0.241782, Accuracy: 92.19
Train Epoch: 49 [37760/50000 (84%)]	Loss: 0.203577, Accuracy: 90.62
Train Epoch: 49 [37824/50000 (84%)]	Loss: 0.135854, Accuracy: 95.31
Train Epoch: 49 [37888/50000 (84%)]	Loss: 0.218922, Accuracy: 90.62
Train Epoch: 49 [37952/50000 (84%)]	Loss: 0.149315, Accuracy: 96.88
Train Epoch: 49 [38016/50000 (84%)]	Loss: 0.291006, Accuracy: 89.06
Train Epoch: 49 [38080/50000 (85%)]	Loss: 0.189833, Accuracy: 95.31
Train Epoch: 49 [38144/50000 (85%)]	Loss: 0.286105, Accuracy: 84.38
Train Epoch: 49 [38208/50000 (85%)]	Loss: 0.181191, Accuracy: 93.75
Train Epoch: 49 [38272/50000 (85%)]	Loss: 0.467306, Accuracy: 89.06
Train Epoch: 49 [38336/50000 (85%)]	Loss: 0.110713, Accuracy: 96.88
Train Epoch: 49 [38400/50000 (85%)]	Loss: 0.2500

NameError: name 'writer' is not defined

In [12]:
test(epoch)


Test set: Average loss: 0.4891, Accuracy: 8565/10000 (85.65%)

